<a href="https://colab.research.google.com/github/0ginny/TIL/blob/main/240401_ANN_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
# 목적은 이 고객이 탈퇴를할지 않할지가 중요한 거야.
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values # 목적 대비 의미있는 모든 데이터
y = dataset.iloc[:,-1].values # 나갔는지 안나갔는지 판단 목적

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
y

array([1, 0, 1, ..., 1, 1, 0])

### Encoding categorical data

Label Encoding the "Gender" column

In [ ]:
# 성별 데이터 int로 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:,2])

In [ ]:
print(X) # female : 0, male : 1

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

[ColumnTrasformer 사용법](https://haehwan.github.io/posts/sta-columntransformer-%EB%B2%88%EC%99%B8/)

In [ ]:
# 이번 인코딩은 조금 복잡하니까 one-hot-encoding 을 사용할 거야
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing  import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder',OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.0,0.0,619,0,42,2,0.0,1,1,1,101348.88
1,0.0,0.0,1.0,608,0,41,1,83807.86,1,0,1,112542.58
2,1.0,0.0,0.0,502,0,42,8,159660.8,3,1,0,113931.57
3,1.0,0.0,0.0,699,0,39,1,0.0,2,0,0,93826.63
4,0.0,0.0,1.0,850,0,43,2,125510.82,1,1,1,79084.1
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,0.0,0.0,771,1,39,5,0.0,2,1,0,96270.64
9996,1.0,0.0,0.0,516,1,35,10,57369.61,1,1,1,101699.77
9997,1.0,0.0,0.0,709,0,36,7,0.0,1,0,1,42085.58
9998,0.0,1.0,0.0,772,1,42,3,75075.31,2,1,0,92888.52


### Splitting the dataset into the Training set and Test set

In [ ]:
# 데이터에서 학습데이터와 검사 데이터를 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0) # 학습 : 검증 = 8 : 2

### Feature Scaling ★★★★★

[feature Scaling 해야하는 이유](https://glanceyes.com/entry/%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%A0%84%EC%B2%98%EB%A6%AC%EC%9D%98-%ED%94%BC%EC%B2%98-%EC%8A%A4%EC%BC%80%EC%9D%BC%EB%A7%81Feature-Scaling)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

[Sequence Model _ tenserFlow _ explain](https://www.tensorflow.org/guide/keras/sequential_model?hl=ko)

[Sequence Model 설명](https://wikidocs.net/192931)

[간단한 모델 설명 및 compile에 대해서](https://minimin2.tistory.com/106)

순차적으로 레이어를 넣기 위해 사용하는 듯함.

In [ ]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

[Dense Layer에 대해서](https://wikidocs.net/192928)

[왜 relu 활성화 함수를 사용할까?](https://for-data-science.tistory.com/68)

In [ ]:
ann.add(tf.keras.layers.Dense( units = 6,# 히든 뉴런의 수, 뭐가 적당할까? 실험적으로 판단하는 수밖에 없어
                              activation = 'relu')) #활성화 함수 지정


### Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense( units = 6, activation ='relu'))

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense( units = 1, # 출력이 (0,1) 이니까 1개면 충분, if (A,B,C) 이런식이라면 3개의 출력뉴런이 필요
                              activation = 'sigmoid')) # 출력에 시그모이드 함수를 쓰면, 맞는지 아닌지와 더불어 1이 될 확률도 구할 수 있어서 출력에선 시그모이드를 더 많이 써.

## Part 3 - Training the ANN

### Compiling the ANN

[Keras 컴파일 종류](https://gggggeun.tistory.com/168)

In [ ]:
ann.compile(optimizer = 'adam', # 확률적 경사 하강법을 쓸 수 있는 optimizer
            loss = 'binary_crossentropy',  # 이진분류에선 반드시 비용함수를 'binary_crossentropy' 로 해줘야해. 그 이상을 분류할 때는 'categorical_crossentropy_loss'
            metrics = 'accuracy' ) # 모니터링할 지표

### Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, # 배치가 있는게 성능적으로 좋아. 하이퍼 파라미터는 32로 주로 사용해.
        epochs = 100) # 반복횟수, 어느정도 커야 결과가 좋아.

Epoch 1/100
250/250 [==============================] - 5s 4ms/step - loss: 0.5825 - accuracy: 0.7475
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4844 - accuracy: 0.7969
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4509 - accuracy: 0.8045
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4356 - accuracy: 0.8114
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4285 - accuracy: 0.8129
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4239 - accuracy: 0.8183
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4187 - accuracy: 0.8179
Epoch 8/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4111 - accuracy: 0.8223
Epoch 9/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4011 - accuracy: 0.8278
Epoch 10/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3894 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

In [ ]:
# my homework code

# 먼저 train 데이터를 확인하자.
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,10,11
0,-1.014607,-0.569844,1.743090,0.169582,-1.091687,-0.464608,0.006661,-1.215717,0.809503,0.642595,-1.032270,1.106432
1,-1.014607,1.754865,-0.573694,-2.304559,0.916013,0.301026,-1.377440,-0.006312,-0.921591,0.642595,0.968738,-0.748664
2,0.985604,-0.569844,-0.573694,-1.191196,-1.091687,-0.943129,-1.031415,0.579935,-0.921591,0.642595,-1.032270,1.485335
3,-1.014607,-0.569844,1.743090,0.035566,0.916013,0.109617,0.006661,0.473128,-0.921591,0.642595,-1.032270,1.276528
4,-1.014607,-0.569844,1.743090,2.056114,-1.091687,1.736588,1.044737,0.810193,0.809503,0.642595,0.968738,0.558378
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,-1.014607,1.754865,-0.573694,-0.582970,-1.091687,-0.656016,-0.339364,0.703104,0.809503,0.642595,0.968738,1.091330
7996,-1.014607,-0.569844,1.743090,1.478815,-1.091687,-1.613058,-0.339364,0.613060,-0.921591,0.642595,0.968738,0.131760
7997,0.985604,-0.569844,-0.573694,0.901515,0.916013,-0.368904,0.006661,1.361474,0.809503,0.642595,-1.032270,1.412320
7998,-1.014607,-0.569844,1.743090,-0.624205,-1.091687,-0.081791,1.390762,-1.215717,0.809503,0.642595,0.968738,0.844321


In [ ]:
# 테스트 데이터는?
pd.DataFrame(X_test)

,0,1,2,3,4,5,6,7,8,9,10,11
0,-1.014607,1.754865,-0.573694,-0.552043,-1.091687,-0.368904,1.044737,0.879303,-0.921591,0.642595,0.968738,1.610857
1,0.985604,-0.569844,-0.573694,-1.314903,-1.091687,0.109617,-1.031415,0.429722,-0.921591,0.642595,-1.032270,0.495870
2,-1.014607,-0.569844,1.743090,0.571630,-1.091687,0.301026,1.044737,0.308583,-0.921591,0.642595,0.968738,-0.424787
3,0.985604,-0.569844,-0.573694,1.416961,0.916013,-0.656016,-0.339364,0.575336,-0.921591,-1.556190,-1.032270,-0.187777
4,-1.014607,1.754865,-0.573694,0.571630,0.916013,-0.081791,0.006661,1.389611,0.809503,0.642595,0.968738,0.616842
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.985604,-0.569844,-0.573694,-0.263393,-1.091687,-1.421650,-1.377440,-1.215717,0.809503,0.642595,0.968738,1.404319
1996,0.985604,-0.569844,-0.573694,-0.665441,-1.091687,-0.368904,0.698712,-1.215717,0.809503,0.642595,-1.032270,-0.511196
1997,-1.014607,-0.569844,1.743090,-0.747912,0.916013,-0.273200,-1.377440,1.297455,0.809503,0.642595,-1.032270,0.718885
1998,-1.014607,1.754865,-0.573694,-0.005670,0.916013,-0.464608,-0.339364,1.059752,-0.921591,0.642595,0.968738,-1.545078


In [ ]:
# 원래 X 데이터는?

pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.0,0.0,619,0,42,2,0.0,1,1,1,101348.88
1,0.0,0.0,1.0,608,0,41,1,83807.86,1,0,1,112542.58
2,1.0,0.0,0.0,502,0,42,8,159660.8,3,1,0,113931.57
3,1.0,0.0,0.0,699,0,39,1,0.0,2,0,0,93826.63
4,0.0,0.0,1.0,850,0,43,2,125510.82,1,1,1,79084.1
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,0.0,0.0,771,1,39,5,0.0,2,1,0,96270.64
9996,1.0,0.0,0.0,516,1,35,10,57369.61,1,1,1,101699.77
9997,1.0,0.0,0.0,709,0,36,7,0.0,1,0,1,42085.58
9998,0.0,1.0,0.0,772,1,42,3,75075.31,2,1,0,92888.52


In [ ]:
# 일단 데이터를 준비해
"""
Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000
"""
test_data = np.array([1,0,0,600,1,40,3,60000.,2,1,1,50000.])

In [ ]:
# 데이터 전처리
test_data.reshape(1,-1) # 예측을위해선 무조건 2차원 [[]] 했으면 사실 이 과정은 필요없어.

array([[1.e+00, 0.e+00, 0.e+00, 6.e+02, 1.e+00, 4.e+01, 3.e+00, 6.e+04,
        2.e+00, 1.e+00, 1.e+00, 5.e+04]])

In [ ]:
# feature scale
test_scale = sc.fit_transform(test_data)

In [ ]:
pd.DataFrame(test_scale)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# predict
ann.predict(test_scale) # 거의 안떠난다

1/1 [==============================] - 0s 65ms/step


array([[0.08974064]], dtype=float32)

**Solution**

In [ ]:
print(ann.predict(sc.fit_transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) > 0.5)

1/1 [==============================] - 0s 105ms/step
[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 5ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [ ]:
# 실제 결과 confusion Matrix로 구하자
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1522   73]
 [ 205  200]]


0.861